In [ ]:
import glob
import pandas as pd
import json
import numpy as np
import difflib

config = json.load(open('AutoEvidence.cfg','r'))
xr_files = glob.glob('gc_test.xls')

ratio_threshold = 0.5

In [ ]:
results = pd.DataFrame({'ID': ['201203317', '201204599', '201300167', '201302420', '201332701', '201401422', '201405845', '201408282', '201408743', '201412349', '201413154', '201415392', '201416907', '201500177', '201500409', '201501004', '208023254', '201503402', '201505812', '201506139', '201508453', '201508659', '201511046', '201512443', '201513000', '208515244', '201515319', '201615948', '201589007', '201600001', '201600463', '201600536', '201600900', '801600911', '201601326', '201601504', '201601513', '201601734', '201602881', '201603007', '201603381', '201604457', '201604818', '201605298', '201606699', '201606820', '201609081', '201609664', '201609799', '201610600', '201702500', '201713954', '201716734', '201726726', '201731106', '201734836', 'M80007273', 'M80007274', 'M80007275', 'M80007276', '450007285', 'M80007287', '4800072M1', 'M80007402'], 'File': ('./test/201203317.pdf', './test/201204599.pdf', './test/201300167.pdf', './test/201302920.pdf', './test/201332701.pdf', './test/201401422.pdf', './test/201405845.pdf', './test/201408282.pdf', './test/201408743.pdf', './test/201412349.pdf', './test/201413154.pdf', './test/201415392.pdf', './test/201416907.pdf', './test/201500177.pdf', './test/201500404.pdf', './test/201501004.pdf', './test/201502325.pdf', './test/201503402.pdf', './test/201505812.pdf', './test/201506139.pdf', './test/201508453.pdf', './test/201508654.pdf', './test/201511046.pdf', './test/201512443.pdf', './test/201513000.pdf', './test/201515244.pdf', './test/201515319.pdf', './test/201515548.pdf', './test/201580007.pdf', './test/201600001.pdf', './test/201600463.pdf', './test/201600536.pdf', './test/201600900.pdf', './test/201600911.pdf', './test/201601326.pdf', './test/201601504.pdf', './test/201601513.pdf', './test/201602734.pdf', './test/201602881.pdf', './test/201603007.pdf', './test/201603381.pdf', './test/201604457.pdf', './test/201604818.pdf', './test/201605298.pdf', './test/201606699.pdf', './test/201606820.pdf', './test/201609081.pdf', './test/201609664.pdf', './test/201609799.pdf', './test/201610600.pdf', './test/201702500.pdf', './test/201713954.pdf', './test/201716734.pdf', './test/201726726.pdf', './test/201731106.pdf', './test/201734836.pdf', './test/M80007273.pdf', './test/M80007274.pdf', './test/M80007275.pdf', './test/M80007276.pdf', './test/M80007285.pdf', './test/M80007287.pdf', './test/M80007291.pdf', './test/M80007402.pdf')})

def load_crossref():
    df = []
    for f in glob.glob('gc_test.xls'):
        df.append(pd.read_csv(f,encoding='utf_16',sep='\t',usecols=[0,1,3]))
        print(f'using crossref: {f}')
    if df == []:
        return None
    else:
        df = pd.concat(df)
        df['ID'] = df['Student ID'].astype(str).map(str.upper)
        df['Name'] = df['First Name'] + ' ' + df['Last Name'].map(lambda x: x.split()[-1])
        df = df.drop(['First Name','Last Name','Student ID'], axis=1)
        df = df.drop_duplicates(subset='ID')
        return df.reset_index(drop=True)


def crossref(results):
    xr = load_crossref()
    results['MatchRatio'] = 0
    if xr is not None:
        results.loc[results.ID.isin(xr.ID),'MatchRatio'] = 1
        xr = xr.drop(xr[xr.ID.isin(results.ID)].index)
        for index, row in results[results.MatchRatio < 1].iterrows():
            ratios = {x:difflib.SequenceMatcher(None, row.ID, x).ratio() for x in xr.ID}
            best = max(ratios, key=ratios.get)
            if ratios[best] >= ratio_threshold:
                print(f'found {best}')
                results.loc[index,'ID'] = best
                results.loc[index,'MatchRatio'] = ratios[best]
    return results

def crossref(results):
    xr = load_crossref()
    results['MatchRatio'] = 0
    results['OldID'] = ''
    results['Name'] = ''
    if xr is not None:
        results.loc[results.ID.isin(xr.ID),'MatchRatio'] = 1
        results = results.merge(xr,how='left',on='ID',suffixes=('_x','')).drop(['Name_x'],axis=1)
        xr = xr.drop(xr[xr.ID.isin(results.ID)].index)
        xr_df = pd.DataFrame(columns=results[results.MatchRatio < 1].ID, index=xr.ID)
        for index, row in results[results.MatchRatio < 1].iterrows():
            xr_df[row.ID] = [difflib.SequenceMatcher(None, row.ID, x).ratio() for x in xr.ID]
        for s_id, maxratio in xr_df.max().sort_values(ascending=False).items():
            newID = xr_df[s_id].idxmax()
            ratio = xr_df[s_id].max()
            if ratio >= ratio_threshold:
                results.loc[results.ID==s_id,'MatchRatio'] = ratio
                results.loc[results.ID==s_id,'Name'] = xr[xr.ID==newID].Name.iloc[0]
                results.loc[results.ID==s_id,'OldID'] = s_id
                results.loc[results.ID==s_id,'ID'] = newID
                xr_df = xr_df.drop(newID)
    return results[['ID','OldID','MatchRatio','Name','File']]

In [ ]:
load_crossref()

In [ ]:
crossref(results)